<a href="https://colab.research.google.com/github/hammad93/hurricane-tts/blob/main/hurricane_tts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation
Please make sure to run all tests.

In [1]:
!git clone https://github.com/hammad93/hurricane-tts.git

Cloning into 'hurricane-tts'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 41 (delta 18), reused 28 (delta 10), pack-reused 0
Receiving objects: 100% (41/41), 13.58 KiB | 6.79 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [2]:
import os
from google.colab import userdata
if not os.getenv("AZURE_OPENAI_API_KEY") :
  os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')

In [3]:
!pip install -r hurricane-tts/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.9/219.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [4]:
%cd hurricane-tts
!python test.py

/content/hurricane-tts
Please note that the AZURE_OPENAI_API_KEY must be set. Here are the constructed messages: [{'role': 'system', 'content': 'You are an AI assistant that helps people find information.'}, {'role': 'user', 'content': 'test'}]
..
----------------------------------------------------------------------
Ran 2 tests in 7.283s

OK


In [10]:
import prompts
# we generate prompts by ingesting live hurricane data and supported languages
storm_data = utils.transform_storm_data()
prompt_data = prompts.generate_prompts()
supported_langs = prompts.unique_lang_list()

In [6]:
# example
print(prompt_data['storms'][0])

These are the input storm records and forecasts.
Each record has a lat and lon according to their geographic coordinates and wind speed in knots.
Please respond with the 3 most relevant languages exactly as they appear from the supported language list other than English each delimited by a comma.
[{'type': 'history', 'lat': 4.8, 'lon': 152.0, 'time': '2023-11-08 18:00:00', 'wind_speed': 15}, {'type': 'history', 'lat': 5.1, 'lon': 151.4, 'time': '2023-11-09 00:00:00', 'wind_speed': 15}, {'type': 'history', 'lat': 5.3, 'lon': 150.8, 'time': '2023-11-09 06:00:00', 'wind_speed': 15}, {'type': 'forecast', 'lat': 5.5, 'lon': 151.0, 'time': '2023-11-09T18:00:00', 'wind_speed': 20}, {'type': 'forecast', 'lat': 5.9, 'lon': 151.5, 'time': '2023-11-10T06:00:00', 'wind_speed': 25}, {'type': 'forecast', 'lat': 6.4, 'lon': 152.0, 'time': '2023-11-10T18:00:00', 'wind_speed': 30}, {'type': 'forecast', 'lat': 6.7, 'lon': 152.5, 'time': '2023-11-11T06:00:00', 'wind_speed': 35}, {'type': 'forecast', 'lat

dict_keys(['WP952023', 'SH912023'])

# Language Geography inference from tropical storm geographical coorindates
We're trying to answer the question, "Which languages do we produce speech for to report on this tropical storm?" This pipeline will produce an English output but also utilize massively multilingual capabilities such that emergency notices are also in local languages

In [15]:
import utils
retries = 5 # sometimes it fails, so we retry it
storm_langs = {} # keys are storm id's and values are the languages
storm_chats = {} # stores chat histories for storms
for index, storm_prompt in enumerate(prompt_data['storms']) :
  storm_id = list(storm_data.keys())[index]
  # Get the languages to generate the report for this storm
  while retries > 0 :
    response = prompts.chat(system=prompt_data['system'], message=storm_prompt)
    print(response)
    result = utils.llm_response_transform(
        resp=response, supported_langs=supported_langs)
    print(result)
    if result :
      break
    else :
      retries = retries - 1
      print(f"Failed. Retries left: {retries}")
  if retries < 1 :
    raise Exception("Couldn't produce a correct output from LLM.")

  # store results
  storm_langs[storm_id] = {'names': result}
  storm_chats[storm_id] = {
      'history' : [{'role': 'system', 'content': prompt_data['system']},
                   {'role': 'user', 'content': storm_prompt},
                   {'role': 'assistant', 'content': response}]
  }

Please note that the AZURE_OPENAI_API_KEY must be set. Here are the constructed messages: [{'role': 'system', 'content': "You are an expert in languages according to their geographical location.\nI have a system for emergency notification of tropical storms that utilizes official data sources and creates speech audio from text with a massively multilingual model.\nThis is the list of supported languages,\n{'Ambai', 'Mixtec, YosondÃºa', 'Spanish', 'Chinantec, OzumacÃ\\xadn', 'Chinese, Hakka', 'Zoque, Francisco LeÃ³n', 'Buli', 'Bodo Parja', 'French', 'Tektiteko', 'Lingao', 'Acholi', 'Chin, Hakha', 'Gilaki', 'Chin, Matu', 'Klon', 'Garo', 'Kandozi-Chapra', 'Molima', 'Quechua, Northern Conchucos Ancash', 'Urat', 'Vietnamese', 'Nagamese', 'Sidamo', 'Mukulu', 'Makonde', 'Somali', 'Gonja', 'Hungarian', 'Chin, Bawm', 'Mato', 'Malayic Dayak', 'Tatuyo', 'Kagulu', 'Palaung, Ruching', 'Binukid', 'Zapotec, San Vicente CoatlÃ¡n', 'Khmu', 'Amazigh', 'Chinantec, Usila', 'Lugbara', 'Gogo', 'Amuzgo, San 

# Tropical Storm Report
This code will create the report in the language specified based on the tropical storm.

In [ ]:
for storm in storm_langs:
  for lang in storm_langs[storm]['names']:
    prompts.chat(message)

The following are from the TTS implementation
https://github.com/facebookresearch/fairseq/blob/main/examples/mms/tts/tutorial/MMS_TTS_Inference_Colab.ipynb


In [ ]:
%cd ~/
!git clone https://github.com/jaywalnut310/vits.git
%cd vits/

!pip install Cython==0.29.21
!pip install librosa==0.8.0
!pip install phonemizer==2.2.1
!pip install scipy
!pip install numpy
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install Unidecode==1.1.1

%cd monotonic_align/
%mkdir monotonic_align
!python3 setup.py build_ext --inplace
%cd ../
%pwd